### Imports 

In [4]:
%matplotlib inline
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import vgg16
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import itertools

### Load & Prep Data

In [5]:
train_path  = './DATA/train'
validation_path  = './DATA/validation'
test_path  = './DATA/test'